In [ ]:
# find the ideal number of clusters with K-means and SpectralClustering
def find_the_ideal_number_of_clusters_kmeans(X, K):
    
    import matplotlib.pyplot as plt
     
    ### Elbow method    
    print("Elbow method:")
    # WCCS and Elbow Method: run several k-means increment k with each iteration, and record the WCCS
    # (WCCS: Within-Cluster-Sum-of-Squares)

    from sklearn.cluster import KMeans


    WCCS=[]
    for i in K:
        kmeans = KMeans(i)
        kmeans.fit(X)
        wcss_iter = kmeans.inertia_
        WCCS.append(wcss_iter)

    number_clusters = K
    plt.plot(number_clusters,WCCS, 'bx-')
    plt.title('The Elbow title')
    plt.xlabel('Number of clusters')
    plt.ylabel('WCSS')
    plt.show()
    
    ### Silhouette coefficient
    print("Silhouette coefficient")
    
    from sklearn.metrics import silhouette_samples, silhouette_score

    silhouette_by_k=[]

    for n_clusters in K:

        # Initialize the clusterer with n_clusters value and a random generator
        # seed of 10 for reproducibility.
        clusterer = KMeans(n_clusters=n_clusters, random_state=1)
        cluster_labels = clusterer.fit_predict(X)

        # The silhouette_score gives the average value for all the samples.
        # This gives a perspective into the density and separation of the formed
        # clusters
        silhouette_avg = silhouette_score(X, cluster_labels)

        silhouette_by_k.append(silhouette_avg)

    plt.plot(K, silhouette_by_k, 'bx-') 
    plt.xlabel('Values of K') 
    plt.ylabel('Silhouette') 
    plt.title('Silhouette method') 
    plt.show() 

    print("Max average silhouette score is :", round(max(silhouette_by_k),5), 
          " for n_clusters =", K[silhouette_by_k.index(max(silhouette_by_k))])
    
    ### Gap statistics
    # With the max value
    print("Gap statistics:")
    from gap_statistic import OptimalK
    from sklearn.cluster import KMeans
    import numpy as np

    optimalK = OptimalK()

    n_clusters = optimalK(X, cluster_array=np.arange(2, 10))
    print('Optimal clusters: ', n_clusters)


    plt.plot(optimalK.gap_df.n_clusters, optimalK.gap_df.gap_value, linewidth=3)
    plt.scatter(optimalK.gap_df[optimalK.gap_df.n_clusters == n_clusters].n_clusters,
                optimalK.gap_df[optimalK.gap_df.n_clusters == n_clusters].gap_value, s=250, c='r')
    plt.grid(True)
    plt.xlabel('Cluster Count')
    plt.ylabel('Gap Value')
    plt.title('Gap Values by Cluster Count')
    plt.show()
    
    # With the minimum value 
    gap_df=optimalK.gap_df
    gap_df.loc[gap_df["diff"]>0].head(1).n_clusters
    print("with minimum value 'method' the optimal number of clusters is:", 
     str(gap_df.loc[gap_df["diff"]>0].head(1).n_clusters.values))

In [ ]:
def fit_kmeans_on_svd(n_clusters, n_components, svd_scores, df_original, plot="yes"):
    
    from sklearn.cluster import KMeans
    import seaborn as sns
    
    # fit data on KMeans
    kmeans_svd=KMeans(n_clusters=n_clusters, init="k-means++", random_state=1)
    kmeans_svd.fit(svd_scores)
    
    # add SVD scores and clusters to the original df
    df_segm_svd_kmeans=pd.concat([df_original.reset_index(drop=True),pd.DataFrame(svd_scores) ], axis=1)
    
    # add SVD components as new columns
    component_columns=list()
    for i in range(n_components):
        component_columns.append("component_"+str(i+1))
    df_segm_svd_kmeans.columns.values[-n_components:]=component_columns
    
    # as last column add the kmeans labels
    df_segm_svd_kmeans["segment_kmeans_svd"]=kmeans_svd.labels_
    df_segm_svd_kmeans["clusters"]=df_segm_svd_kmeans["segment_kmeans_svd"]+1
    
    if plot=="yes":
        # Plot data
        x_axis=df_segm_svd_kmeans.component_2
        y_axis=df_segm_svd_kmeans.component_1
        plt.figure(figsize=(10,8))
        sns.scatterplot(x_axis, y_axis,
                       hue=df_segm_svd_kmeans.clusters,
                        palette="Set2")
        plt.title("Clusters by SVD components")
        plt.legend(title='Clusters')
        plt.show()
        
        return df_segm_svd_kmeans
    else:   
        return df_segm_svd_kmeans

    

In [1]:
def fit_transform_svd(n_components, scaled_data):
    from sklearn.decomposition import TruncatedSVD, PCA
    
    svd = TruncatedSVD(n_components)
    svd.fit(scaled_data)
    return svd.transform(scaled_data)

In [ ]:
# get dummies
def get_dummies(df,columns):
    import pandas as pd
    return pd.get_dummies(data=df, columns=columns, drop_first=True)

In [ ]:
def svd_optimal_dimensions(data):
    
    import matplotlib.pyplot as plt

    # SVD for the scaled data
    Ul, Dl, Vt = np.linalg.svd(data)
    Vl = Vt.transpose()
    # print the new coordinate vectors:
    print("New coordinate vectors")
    plt.plot(Dl)
    plt.show()

    # explained variance
    Zlext_more = np.matmul(Ul[:,0:10],np.diag(Dl[0:10]))
    df_explained = pd.DataFrame(Zlext_more)
    df_explained.var().plot(marker='o', linestyle="--",
                            title="Explained variance") # This drop off provides us an indicator of how many to pick 
    plt.show()
    
    # Error of low dimensional representation
    print("Optimal low-rank approximation")
    rmax = data.shape[1]+1
    errors = np.zeros(rmax-1)
    r = range(1,rmax)
    n=data.shape[0]
    p=data.shape[1]
    for i in r:
        Zext_temp = np.matmul(Ul[:,0:i],np.diag(Dl[0:i]))
        Xapprox = np.matmul(Zext_temp,np.transpose(Vl[:,0:i]))
        resid = data-Xapprox
        residsq = np.matmul(resid.transpose(),resid)
        errors[i-1] = residsq.trace()
    #plt.plot(r,errors)
    plt.plot(r,errors/(n*p)) # errros per point
    plt.xlabel('Final low-rank dimensions')
    plt.ylabel('Error of low dimensional representation')
    #plt.title("Optimal low-rank approximation")
    plt.show()

    # Drop in error
    print("Drop in error")
    plt.plot(r[1:],abs(np.diff(errors/(n*p))),marker='o', linestyle="--")
    plt.xticks(np.arange(min(r[1:]),data.shape[1]+1 , 1.0))
    plt.xlabel('Final low-rank dimensions')
    plt.ylabel('Drop in error of low dimensional representation')
    #plt.title("Drop in error")
    plt.show()

In [ ]:
def remap_icd9_codes(x):
    if (x >= 1 and x<=139):
        return 'infectious and parasitic diseases'
    elif (x >= 140 and x<=239):
        return 'neoplasms'
    elif (x >= 240 and x<=279):
        return 'endocrine, nutritional and metabolic diseases, and immunity disorders'
    elif (x >= 280 and x<= 289):
        return 'diseases of the blood and blood-forming organs'
    elif (x >= 290 and x<=319):
        return 'mental disorders'
    elif (x >= 320 and x<=389):
        return 'diseases of the nervous system and sense organs'
    elif (x >= 390 and x<=459):
        return 'diseases of the circulatory system' 
    elif (x >= 460 and x<=519):
        return 'diseases of the respiratory system'
    
    elif (x >= 520 and x<=579):
        return 'diseases of the digestive system'
        
    elif (x >= 580 and x<=629):
        return 'diseases of the genitourinary system'
        
    elif (x >= 630 and x<=679):
        return 'complications of pregnancy, childbirth, and the puerperium'
        
    elif (x >= 680 and x<=709):
        return 'diseases of the skin and subcutaneous tissue'
        
    elif (x >= 710 and x<=739):
        return 'diseases of the musculoskeletal system and connective tissue'
        
    elif (x >= 740 and x<=759):
        return 'congenital anomalies'
        
    elif (x >= 710 and x<=779):
        return 'certain conditions originating in the perinatal period'
        
    elif (x >= 780 and x<=799):
        return 'symptoms, signs, and ill-defined conditions'
        
    elif (x >= 800 and x<=999):
        return "injury and poisoning"
    else:
        return 'external causes of injury and supplemental classification'


In [ ]:
def most_common_diagnoses_by_clusters(diagnoses_responses, cluster):
    df=pd.DataFrame(diagnoses_responses.groupby([cluster, 'icd9_groups_conc']).size()).reset_index()

    df.sort_values(by=0, ascending=False, inplace=True)

    df=df.groupby(cluster).head(3).sort_values(by=cluster)
    return df
    

In [ ]:
def outcomes_barplots(diagnoses_df, cluster_col, category_col):
    mean = lambda x: x.mean()
    outcomes_by=diagnoses_responses.groupby([cluster_col, 
                                                 category_col])["los", 
                                                                    "diagnosis_count", 
                                                                    "hospital_expire_flag"].agg(mean).reset_index()

    outcomes_by=outcomes_by.rename(columns={"hospital_expire_flag": "death_rate"})

    # Visualize results by age
    sns.set_theme(style="whitegrid")
    sns.catplot(x=category_col, col=cluster_col, y="death_rate",
                     kind="bar", data=outcomes_by, palette="Set2")

    

In [ ]:
## the correction factor: 
def reweight(pi,q1,r1):
    r0 = 1-r1
    q0 = 1-q1
    tot = pi*(q1/r1)+(1-pi)*(q0/r0)
    w = pi*(q1/r1)
    w /= tot
    return w

In [ ]:
def predict_and_evaluate_binary(X,y, model, crossval="Yes"):
    
    from sklearn.linear_model import LogisticRegression
    from sklearn import metrics
    from sklearn.metrics import confusion_matrix
    import pandas as pd
    
    model.fit(X,y)
    y_hat=model.predict_proba(X)
    
    # reweighting 
    q1 = y.sum()/len(y)
    r1 = 0.5
    y_hat_corr=reweight(y_hat[:,1], q1,r1)
    
    
    ### Evaluate Model ###
    y_pred_new = [1 if pi >= 0.25 else 0 for pi in y_hat_corr]
    
    # Confusion Matrix
    print("Confusion Matrix \n")
    # insample_labels = model.predict(X)
    cm =  confusion_matrix(y_pred=y_pred_new, y_true=y, labels=[0,1])
    print (cm)
    
    # Plotting confusion matrix (custom help function)
    df_cm = pd.DataFrame(cm, index = [i for i in class_labels],
                  columns = [i for i in class_labels])
    sns.set(font_scale=1)
    sns.heatmap(df_cm, annot=True, fmt='g', cmap='Blues')
    plt.xlabel("Predicted label")
    plt.ylabel("Real label")
    plt.show()
    
    # ROC AUC score
    #get_auc(y_original,y_hat_corr , class_labels, column=1, plot=True) 
    fpr, tpr, _ = metrics.roc_curve(y == 1, y_hat_corr,drop_intermediate = False)
    roc_auc = metrics.roc_auc_score(y_true=y, y_score=y_hat_corr)
    print ("AUC: ", roc_auc)
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()
    

    # Classification report
    print("Classification report of the model: \n", 
      metrics.classification_report(y,y_pred_new))
    
    if crossval=="Yes":
        # cross validation
        print("Confusion matrix of in-sample cross-validation:")

        from sklearn.model_selection import cross_val_predict as cvp
        y_hat_cv = cvp(model, X, y, cv=100)

        cm2 =  confusion_matrix(y_pred=y_hat_cv, y_true=y, labels=[0,1])
        #print(cm2)
        df_cm2 = pd.DataFrame(cm2, index = [i for i in class_labels],
                      columns = [i for i in class_labels])
        sns.set(font_scale=1)
        sns.heatmap(df_cm, annot=True, fmt='g', cmap='Blues')
        plt.xlabel("Predicted label")
        plt.ylabel("Real label")
        plt.show()
        
    elif crossval=="No":
        print("No cross-validation")
